In [1]:
import math
import pickle
import numpy as np
import tensorflow as tf
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten   
from tensorflow.keras.layers import Layer, ZeroPadding2D, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler

### Helpers

In [2]:
# an implication of Pytorch CrossMapLRN2d with Keras
class LRN2D(Layer):
    """
    This code is adapted from pylearn2.
    License at: https://github.com/lisa-lab/pylearn2/blob/master/LICENSE.txt
    """

    def __init__(self, alpha=1e-4, k=2, beta=0.75, n=5):
        if n % 2 == 0:
            raise NotImplementedError('LRN2D only works with odd n. n provided: ' + str(n))
        super(LRN2D, self).__init__()
        self.alpha = alpha
        self.k = k
        self.beta = beta
        self.n = n

    def get_output(self, train):
        X = self.get_input(train)
        b, ch, r, c = X.shape
        half_n = self.n // 2
        input_sqr = T.sqr(X)
        extra_channels = T.alloc(0., b, ch + 2*half_n, r, c)
        input_sqr = T.set_subtensor(extra_channels[:, half_n:half_n+ch, :, :], input_sqr)
        scale = self.k
        for i in range(self.n):
            scale += self.alpha * input_sqr[:, i:i+ch, :, :]
        scale = scale ** self.beta
        return X / scale

    def get_config(self):
        return {'name': self.__class__.__name__,
                'alpha': self.alpha,
                'k': self.k,
                'beta': self.beta,
                'n': self.n}

    

# another implication of Pytorch CrossMapLRN2d with Keras
class LocalResponseNormalization(Layer):
  
    def __init__(self, n=5, alpha=1e-4, beta=0.75, k=2, **kwargs):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.k = k
        super(LocalResponseNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.shape = input_shape
        super(LocalResponseNormalization, self).build(input_shape)

    def call(self, x):
        _, r, c, f = self.shape 
        squared = K.square(x)
        pooled = K.pool2d(squared, (self.n, self.n), strides=(1,1), padding='same', pool_mode='avg')
        summed = K.sum(pooled, axis=3, keepdims=True)
        averaged = self.alpha * K.repeat_elements(summed, f, axis=3)
        denom = K.pow(self.k + averaged, self.beta)
        return x / denom 
    
    def compute_output_shape(self, input_shape):
        return input_shape



# Function for changing learning rate over epochs
def lr_decay(epoch):
    decay = .1
    lr = base_lr * math.pow(decay, (epoch // 30))   
    return lr


## **1. Data**

In [11]:
# load the scaled dataset
path = 'D:/ETE/Data/Data_Mini/pickles/'
pickles = ['nose', 'left_eye', 'right_eye', 'face', 'grid_paper', 'labels']

for pkl in tqdm(pickles):
    with open(path + pkl + '.pkl','rb') as f:
        exec(pkl + ' = pickle.load(f)')

## **2. The Model**

### **Eye Nose**

In [3]:
nose_input = Input(shape=(64,64,3))
nose_cnn1_out = Conv2D(filters=96,
                       kernel_size=11,
                       strides=4,
                       padding='valid',
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4),
                       input_shape=(224,224,3))(nose_input)
nose_max1_out = MaxPool2D(pool_size=3, strides=2)(nose_cnn1_out)
nose_lrn1_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(nose_max1_out)
nose_zro1_out = ZeroPadding2D(padding=2)(nose_lrn1_out)
nose_cnn2_out = Conv2D(filters=256,
                       kernel_size=5,
                       strides=1,
                       padding='valid',
                       #groups=2,
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4))(nose_zro1_out)
nose_max2_out = MaxPool2D(pool_size=3, strides=2)(nose_cnn2_out)
nose_lrn2_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(nose_max2_out)
nose_cnn3_out = Conv2D(filters=384,
                       kernel_size=3,
                       strides=1,
                       padding='same',
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4))(nose_lrn2_out)
nose_cnn4_out = Conv2D(filters=64,
                       kernel_size=1,
                       strides=1,
                       padding='valid',
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4))(nose_cnn3_out)
nose_flt_out = Flatten()(nose_cnn4_out)
nose_dns_out = Dense(128, activation = 'relu',
                     kernel_regularizer=regularizers.l2(1e-4))(nose_flt_out)
nose_out = Dense(64, activation = 'relu')(nose_dns_out)


### **Eye Net**

In [4]:
eye_input = Input(shape=(64,64,3))
eye_cnn1_out = Conv2D(filters=96,
                       kernel_size=11,
                       strides=4,
                       padding='valid',
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4),
                       input_shape=(224,224,3))(eye_input)
eye_max1_out = MaxPool2D(pool_size=3, strides=2)(eye_cnn1_out)
eye_lrn1_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(eye_max1_out)
eye_zro1_out = ZeroPadding2D(padding=2)(eye_lrn1_out)
eye_cnn2_out = Conv2D(filters=256,
                      kernel_size=5,
                      strides=1,
                      padding='valid',
                      #groups=2,
                      activation='relu',
                      kernel_regularizer=regularizers.l2(1e-4))(eye_zro1_out)
eye_max2_out = MaxPool2D(pool_size=3, strides=2)(eye_cnn2_out)
eye_lrn2_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(eye_max2_out)
eye_cnn3_out = Conv2D(filters=384,
                      kernel_size=3,
                      strides=1,
                      padding='same',
                      activation='relu',
                      kernel_regularizer=regularizers.l2(1e-4))(eye_lrn2_out)
eye_cnn4_out = Conv2D(filters=64,
                      kernel_size=1,
                      strides=1,
                      padding='valid',
                      activation='relu',
                      kernel_regularizer=regularizers.l2(1e-4))(eye_cnn3_out)
eye_out = Flatten()(eye_cnn4_out)

model_eye = Model(eye_input, eye_out)


### **Face Net**

In [5]:
face_input = Input(shape=(64,64,3))
face_cnn1_out = Conv2D(filters=96,
                       kernel_size=11,
                       strides=4,
                       padding='valid',
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4),
                       input_shape=(224,224,3))(face_input)
face_max1_out = MaxPool2D(pool_size=3, strides=2)(face_cnn1_out)
face_lrn1_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(face_max1_out)
face_zro1_out = ZeroPadding2D(padding=2)(face_lrn1_out)
face_cnn2_out = Conv2D(filters=256,
                       kernel_size=5,
                       strides=1,
                       padding='valid',
                       #groups=2,
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4))(face_zro1_out)
face_max2_out = MaxPool2D(pool_size=3, strides=2)(face_cnn2_out)
face_lrn2_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(face_max2_out)
face_cnn3_out = Conv2D(filters=384,
                       kernel_size=3,
                       strides=1,
                       padding='same',
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4))(face_lrn2_out)
face_cnn4_out = Conv2D(filters=64,
                       kernel_size=1,
                       strides=1,
                       padding='valid',
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4))(face_cnn3_out)
face_flt_out = Flatten()(face_cnn4_out)
face_dns_out = Dense(128, activation = 'relu',
                     kernel_regularizer=regularizers.l2(1e-4))(face_flt_out)
face_out = Dense(64, activation = 'relu')(face_dns_out)

### **Papaer Grid Net**

In [6]:
grid_input = Input(shape=(25,25))
grid_flt_out = Flatten()(grid_input)
grid_dns_out = Dense(256, activation = 'relu',
                     kernel_regularizer=regularizers.l2(1e-4))(grid_flt_out)
grid_out = Dense(128, activation = 'relu',
                 kernel_regularizer=regularizers.l2(1e-4))(grid_dns_out)


### **Merge the Eyes' Nets**

In [7]:
eye_left_input = Input(shape=(64,64,3))
eye_right_input = Input(shape=(64,64,3))
eye_left_out = model_eye(eye_left_input)
eye_right_out = model_eye(eye_right_input)

concat_eyes = concatenate([eye_left_out, eye_right_out])

eyes_out = Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(1e-4))(concat_eyes)


### **Merge the Eyes, Face, and the Grid Nets**

In [8]:
concat_all = concatenate([nose_out, eyes_out, face_out, grid_out])

final_dns_out = Dense(128, activation='relu',
                      kernel_regularizer=regularizers.l2(1e-4))(concat_all)
final_out = Dense(2, activation='relu',
                  kernel_regularizer=regularizers.l2(1e-4))(final_dns_out)

final_model = Model([nose_input, eye_left_input, eye_right_input, face_input, grid_input], final_out)


In [9]:
final_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 14, 14, 96)   34944       input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 14, 14, 96)   34944       input_3[0][0]                    
____________________________________________________________________________________________

## **3. Training**

In [10]:
lr_callback = LearningRateScheduler(lr_decay)
checkpointer = ModelCheckpoint(filepath='Weights.hdf5', monitor='val_loss',
                               verbose=1, save_best_only=True)
base_lr = 0.0001
momentum = 0.9

# sgd = SGD(lr=base_lr, momentum=momentum)
sgd = SGD(lr=1e-1, decay=5e-4, momentum=9e-1, nesterov=True)
adam = Adam(lr=1e-1)

In [13]:
final_model.compile(optimizer=sgd, loss='mse', metrics=['mae'])

final_model.fit(x=[nose[:6000], left_eye[:6000], right_eye[:6000], face[:6000], grid_paper[:6000]], 
          y=labels[:6000],
          batch_size=64,
          epochs=25,
          validation_split=0.2,
          shuffle=True,
          callbacks=[lr_callback, checkpointer])


Train on 4800 samples, validate on 1200 samples
Epoch 1/25
4672/4800 [============================>.] - ETA: 0s - loss: 31.3140 - mae: 4.0825
Epoch 00001: val_loss did not improve from 27.50179
4800/4800 [==============================] - 5s 991us/sample - loss: 31.2404 - mae: 4.0773 - val_loss: 27.5043 - val_mae: 3.8657
Epoch 2/25
4672/4800 [============================>.] - ETA: 0s - loss: 31.1260 - mae: 4.0734
Epoch 00002: val_loss did not improve from 27.50179
4800/4800 [==============================] - 3s 568us/sample - loss: 31.2395 - mae: 4.0771 - val_loss: 27.5063 - val_mae: 3.8662
Epoch 3/25
4672/4800 [============================>.] - ETA: 0s - loss: 31.3410 - mae: 4.0839 ETA: 1s - loss: 30.7004 - mae:  - ETA: 1s - loss: 31.3305 - ETA: 0s - loss: 31.42
Epoch 00003: val_loss did not improve from 27.50179
4800/4800 [==============================] - 3s 571us/sample - loss: 31.2399 - mae: 4.0768 - val_loss: 27.5068 - val_mae: 3.8661
Epoch 4/25
4672/4800 [=======================